In [ ]:
##### Download all CUDEM tiles #####
####################################

import os
import urllib

def get_filename(url):
    """
    Parses filename from given url
    """
    if url.find('/'):
        return url.rsplit('/', 1)[1]

# Filepaths
outdir = r"E:\CUDEM"

# File locations in txt file (downloaded from https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/ - URL list)
with open(r'E:\CUDEM\Tiles.txt') as f:
    flat_list=[word for line in f for word in line.split()]
    
url_list = flat_list

# Create folder if it does no exist
if not os.path.exists(outdir):
    os.makedirs(outdir)

# Download files
for url in url_list:
    # Parse filename
    fname = get_filename(url)
    outfp = os.path.join(outdir, fname)
    # Download the file if it does not exist already
    if not os.path.exists(outfp):
        print("Downloading", fname)
        r = urllib.request.urlretrieve(url, outfp)

In [ ]:
# # Check raster crs

# import gdal
# import osr
# raster = gdal.Open(r'\\snamResearch.files.soton.ac.uk\snamResearch\A3_BARRIERS_DEVELOPMENT\NC_clip\Mosaic_Clip.tif')


# proj = osr.SpatialReference(wkt=raster.GetProjection())
# print(proj.GetAttrValue('AUTHORITY',1))

In [ ]:
# # Change all rasters to other crs

# from osgeo import gdal
# import os
# # Loop within folder
# rootdir = 'C:\\...\\'
# extensions = ('.tif')

# for subdir, dirs, files in os.walk(rootdir):
#     for file in files:
#         ext = os.path.splitext(file)[-1].lower()
#         if ext in extensions:
#             print (os.path.join(subdir, file))
#             file_path = os.path.join(subdir, file)
#             input_raster = file_path
#             output_raster= 'C:\\...\\...\\{0}.tif'.format(file)
#             ds = gdal.Warp(outputfile, inputfile, dstSRS='EPSG:4269')
#             warp = None # Closes the files

In [ ]:
##### Check if raster and NC barrier overlap, if true, clip raster using polygon and save it #####
##################################################################################################

import ogr, gdal
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import os

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

for filename1 in os.listdir(r'E:\CUDEM'):
    if filename1.endswith('.tif'):
        raster_dir=(r'E:\CUDEM\{0}'.format(filename1))
        raster_name=filename1
        raster = gdal.Open(raster_dir)
        #print(raster_dir)
        # Get raster geometry
        transform = raster.GetGeoTransform()
        pixelWidth = transform[1]
        pixelHeight = transform[5]
        cols = raster.RasterXSize
        rows = raster.RasterYSize

        xLeft = transform[0]
        yTop = transform[3]
        xRight = xLeft+cols*pixelWidth
        yBottom = yTop+rows*pixelHeight

        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint(xLeft, yTop)
        ring.AddPoint(xLeft, yBottom)
        ring.AddPoint(xRight, yBottom)
        ring.AddPoint(xRight, yTop)
        ring.AddPoint(xLeft, yTop)
        rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
        rasterGeometry.AddGeometry(ring)
        
        for filename2 in os.listdir('C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons\\'):
            if filename2.endswith('.shp'):
                vector_dir=('C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons\\{0}'.format(filename2))
                vector_name=filename2
                vector = ogr.Open(vector_dir)
                #print(vector_dir)
                        
                # Get vector geometry
                layer = vector.GetLayer()
                feature = layer.GetFeature(0)
                vectorGeometry = feature.GetGeometryRef()

                #print(rasterGeometry.Intersect(vectorGeometry))
                
                if  rasterGeometry.Intersect(vectorGeometry) == True:
                        print('True')
                        # Output raster
                        out_tif = os.path.join(r'E:\CUDEM_CLIP\{0}_{1}.tif'.format(vector_name,raster_name))
                        # Read the data
                        data = rasterio.open(raster_dir)
                        barrier = gpd.read_file(vector_dir)
                        # Project the Polygon into same CRS as the grid
                        barrier = barrier.to_crs(crs=data.crs.data)
                        coords = getFeatures(barrier)
                        # Clip the raster with Polygon
                        out_img, out_transform = mask(dataset=data, shapes=coords, crop=True)
                        # Copy the metadata
                        out_meta = data.meta.copy()
                        # Parse EPSG code
                        epsg_code = int(data.crs.data['init'][5:])
                        out_meta.update({"driver": "GTiff",
                                        "height": out_img.shape[1],
                                        "width": out_img.shape[2],
                                        "transform": out_transform,
                                        "crs": "EPSG:4269"}
                                                 )
                        with rasterio.open(out_tif, "w", **out_meta) as dest:
                            dest.write(out_img)
                        print(out_tif)
                            
                else:
                    continue

In [ ]:
##### Create mosaic for each barrier - North Carolina #####
###########################################################

# Import packages
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

for vector in os.listdir('C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons\\'):
            if vector.endswith('.shp'):
                vector_name= vector.replace('.shp', '')
                # List for the source files
                src_files_to_mosaic = []
                for raster in os.listdir(r'E:\CUDEM_CLIP'):
                            if raster.startswith(vector_name):
                                src = rasterio.open(r'E:\CUDEM_CLIP\{0}'.format(raster))
                                src_files_to_mosaic.append(src)
                                # Merge function returns a single mosaic array and the transformation info
                                mosaic, out_trans = merge(src_files_to_mosaic)
                                # Copy the metadata
                                out_meta = src.meta.copy()
                                # Update the metadata
                                out_meta.update({"driver": "GTiff",
                                                 "height": mosaic.shape[1],
                                                 "width": mosaic.shape[2],
                                                 "transform": out_trans,
                                                 "crs": "EPSG:4269"
                                                 }
                                                )

                                # Write the mosaic raster to disk
                                with rasterio.open(r"E:\CUDEM_CLIP\Mosaic{0}.tif".format(vector_name), "w", **out_meta) as dest:
                                    dest.write(mosaic)
                                print(vector_name)
                            else:
                                continue
                                                
              

In [ ]:
##### Extract elevation data for nodes - all NC barriers #####
##############################################################

%matplotlib inline
import osmnx as ox
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
import numpy as np
import ogr, gdal
from shapely.geometry import box
from fiona.crs import from_epsg
import pycrs


for filename1 in os.listdir(r'E:\NC_CUDEM\MOSAIC'):
    if filename1.endswith('.tif'):
        raster_dir=(r'E:\NC_CUDEM\MOSAIC\{0}'.format(filename1))
        raster_name=filename1
        raster = gdal.Open(raster_dir)
        print(raster_name)
        # Get raster geometry
        transform = raster.GetGeoTransform()
        pixelWidth = transform[1]
        pixelHeight = transform[5]
        cols = raster.RasterXSize
        rows = raster.RasterYSize

        xLeft = transform[0]
        yTop = transform[3]
        xRight = xLeft+cols*pixelWidth
        yBottom = yTop+rows*pixelHeight

        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint(xLeft, yTop)
        ring.AddPoint(xLeft, yBottom)
        ring.AddPoint(xRight, yBottom)
        ring.AddPoint(xRight, yTop)
        ring.AddPoint(xLeft, yTop)
        rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
        rasterGeometry.AddGeometry(ring)
        
        for filename2 in os.listdir('C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons\\'):
            if filename2.endswith('.shp'):
                vector_dir=('C:\\Users\\Sofia\\Dropbox\\Barriers_US\\NorthCarolina\\Polygons\\{0}'.format(filename2))
                vector_name=filename2
                vector_name=vector_name.replace('.shp','')
                vector = ogr.Open(vector_dir)
                #print(vector_dir)
                        
                # Get vector geometry
                layer = vector.GetLayer()
                feature = layer.GetFeature(0)
                vectorGeometry = feature.GetGeometryRef()
               
                if  rasterGeometry.Intersect(vectorGeometry) == True:
                    try:
                        print('True')
                        print(vector_name)
                        # Read polygons
                        poly = gpd.read_file(vector_dir)
                        # Extract just the geometry (shapely object) part
                        poly_geo = poly['geometry'].iloc[0]
                        # Clean it with a buffer
                        poly_geo = poly_geo.buffer(0)
                        poly_geo.is_valid
                        G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
                        # Retrieve nodes and edges
                        nodes, edges = ox.graph_to_gdfs(G)
                        # Create an index for the geodataframe nodes
                        nodes['index'] = range(0, len(nodes))
                        # Convert to same crs 
                        nodes = nodes.to_crs("epsg:4269") 

                        # Open the raster and store metadata

                        raster= rasterio.open(raster_dir)
                        # Extract point value from raster
                        values=[]

                        for point in nodes['geometry']:
                            x = point.xy[0][0]
                            y = point.xy[1][0]
                            row, col=raster.index(x,y)
                            #print('Point correspond to row, col: %d, %d'%(row,col))
                            value=raster.read(1)[row,col]
                            #print('Raster value on point %.2f \n'%value)
                            values.append(value)

                        nodes['elevation']=values
                        nodes.to_file(r"E:\NC_CUDEM\Nodes\{0}.shp".format(vector_name))
                    except:
                        continue

In [ ]:
##### Extract elevation data for nodes - Just one barrier #####
##########################################################

# %matplotlib inline
# import osmnx as ox
# import matplotlib.pyplot as plt
# import geopandas as gpd
# import rasterio
# from rasterio.plot import show
# import numpy as np
# # Read polygons
# poly = gpd.read_file("C:\\Users\\Sofia\\Dropbox\\Barriers_US\\Polygons\\NC10_geo.shp")

# # Extract just the geometry (shapely object) part
# poly_geo = poly['geometry'].iloc[0]
# # Clean it with a buffer
# poly_geo = poly_geo.buffer(0)
# poly_geo.is_valid
# G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
# # Retrieve nodes and edges
# nodes, edges = ox.graph_to_gdfs(G)
# # Create an index for the geodataframe nodes
# nodes['index'] = range(0, len(nodes))
# # Convert to same crs 
# nodes = nodes.to_crs("epsg:4269") 

# # Open the raster and store metadata

# raster= rasterio.open(r"E:\NC_CUDEM\MOSAIC\MosaicNC11.tif")
    

# # Extract point value from raster
# values=[]

# for point in nodes['geometry']:
#     x = point.xy[0][0]
#     y = point.xy[1][0]
#     row, col=raster.index(x,y)
#     print('Point correspond to row, col: %d, %d'%(row,col))
#     value=raster.read(1)[row,col]
#     print('Raster value on point %.2f \n'%value)
#     values.append(value)

# nodes['elevation']=values
# nodes.head()